In [1]:
import findspark
from memory_profiler import profile
findspark.init('/home/Ilan/miniconda/envs/garage/lib/python2.7/site-packages/pyspark')
findspark.add_jars('/usr/local/bin/mysql-connector-java-5.1.46-bin.jar')
from pyspark import SparkContext , SQLContext
from pyspark.sql import SparkSession
from Trax.Utils.Conf.Configuration import Config
from Trax.Data.Projects.Connector import ProjectConnector
from Trax.Cloud.Services.Connector.Keys import DbUsers
from Trax.Data.Projects.Connector import ProjectConnector
import pandas as pd


#sc = SparkContext(appName="myAppName")
spark = SparkSession.builder.appName("run_etl").config("spark.driver.memory","4g").config("spark.executor.memory", "4g").config("spark.driver.cores", "4").config("spark.driver.maxResultSize", "4").getOrCreate()



### project params 

In [8]:
project_name = 'jnjuk'
env = 'prod'
start_date = '2018-07-01'
end_date =  '2018-07-10'
batch_size = 80000

In [3]:
from Trax.Utils.Conf.Configuration import Config
from Trax.Data.Projects.Connector import ProjectConnector
from Trax.Cloud.Services.Connector.Keys import DbUsers

Config.init(app_name=' ', default_env=env ,config_file='~/theGarage/Trax/Apps/Services/KEngine/k-engine-prod.config')
Config.PROD

connector = ProjectConnector(project_name,DbUsers.CalculationEng)

Log was not initialized - get_credentials: get request returned errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - get_credentials: get request returned errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - get_credentials: Failed to retrieve credentials from vault: exception is :Login to vault failed 1 tiems. last login resulted errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - Traceback (most recent call last):
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/Credentials.py", line 89, in get_credentials
    token = cls.cloud_adapter.get_vault_token(force)
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/ConnectorAdapter.py", line 94, in get_vault_token
    self._login_vault_request()
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/ConnectorAdapter.py", line 70, in _login_vault_request
    self._check_for_

In [5]:
connector.project_params['rds_name']
project_url = 'jdbc:mysql://{}/report'.format(connector.project_params['rds_name'])

results = '''    
                    (SELECT 
                        report.kpi_level_2_results.*
                    FROM
                        report.kpi_level_2_results,
                        probedata.session
                    WHERE
                        probedata.session.pk = report.kpi_level_2_results.session_fk
                            AND probedata.session.visit_date BETWEEN '{}' AND '{}')tmp_kpi_level_2_results '''.format(start_date, end_date)

static = '''(SELECT * FROM  static.kpi_level_2 where static.kpi_level_2.kpi_calculation_stage_fk = 3) static_kpi '''

In [6]:
connector = ProjectConnector('diageous',DbUsers.CalculationEng)
meta_results = '''    
            SELECT 
                count(*) count, 
                MIN(probedata.session.pk) min,
                MAX(probedata.session.pk) max
            FROM
                report.kpi_level_2_results,
                probedata.session
            WHERE
                probedata.session.pk = report.kpi_level_2_results.session_fk
                    AND probedata.session.visit_date BETWEEN '{}' AND '{}';'''.format(start_date, end_date)
kpi_results_meta_data = pd.read_sql_query(meta_results, connector.db)
connector.disconnect_rds()

Log was not initialized - get_credentials: get request returned errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - get_credentials: get request returned errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - get_credentials: Failed to retrieve credentials from vault: exception is :Login to vault failed 1 tiems. last login resulted errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - Traceback (most recent call last):
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/Credentials.py", line 89, in get_credentials
    token = cls.cloud_adapter.get_vault_token(force)
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/ConnectorAdapter.py", line 94, in get_vault_token
    self._login_vault_request()
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/ConnectorAdapter.py", line 70, in _login_vault_request
    self._check_for_

No handlers could be found for logger "Trax"


In [9]:
lowerBound = kpi_results_meta_data.loc[0]['min']
upperBound = kpi_results_meta_data.loc[0]['max']
count_of_row = kpi_results_meta_data.loc[0]['count']

if (count_of_row / batch_size ) > 1 :
    number_of_partition = (count_of_row / batch_size )
else: 
    number_of_partition = 1 
print "count of rows : "  + str(count_of_row)    
print " number of partition:" + str(number_of_partition)

count of rows : 2605325
 number of partition:32


In [10]:
static_kpi = spark.read.jdbc(url=project_url,
                          table=static,                          
                          properties={"user": connector.dbuser.username, 
                                      "password":connector.dbuser.cred,
                                      "driver": 'com.mysql.jdbc.Driver'}).cache()


static_kpi.count()


11

In [11]:
kpi_results = spark.read.jdbc(url=project_url,
                          table=results,                          
                          properties={"user": connector.dbuser.username, 
                                      "password":connector.dbuser.cred,
                                      "partitionColumn":"tmp_kpi_level_2_results.session_fk" ,
                                      "lowerBound":"{}".format(lowerBound),
                                      "upperBound":"{}".format(upperBound),
                                      "numPartitions":"{}".format(number_of_partition),
                                      "driver": 'com.mysql.jdbc.Driver'}).cache()

kpi_results.count()


6191753

### test 1 - kpi not calculated 

In [13]:
merged_kpi_results = static_kpi.join(kpi_results, static_kpi.pk == kpi_results.kpi_level_2_fk ,how='left')
# merged_kpi_results.select("session_fk","client_name","result").filter('result is null').show()
merged_kpi_results.select("client_name").filter('result is null').groupBy("client_name").count().show()

+------------------+-----+
|       client_name|count|
+------------------+-----+
|eye_hand_level_sos|  313|
+------------------+-----+



### test -invalid % results

In [17]:
#merged_kpi_results.select("session_fk","client_name","result").filter('is_percent = 1 and result < 0 or result > 1').show(1000 ,False)
merged_kpi_results.select("client_name").filter('is_percent = 1 and result < 0 or result > 1').groupBy("client_name").count().show(1000 ,False)


+-----------------------------------+-----+
|client_name                        |count|
+-----------------------------------+-----+
|promo                              |198  |
|PRODUCT_PRESENCE_IN_CATEGORY_CUSTOM|15883|
|store_assortment - SKU             |15883|
|eye_hand_level_sos                 |10173|
|promo - SKU                        |5172 |
+-----------------------------------+-----+



### test -results with value 0 


In [18]:
import pyspark.sql.functions as F

In [19]:
results_with_zero2 = merged_kpi_results.select("session_fk","client_name","result").filter( 'result == 0')


merged_kpi_results2 = merged_kpi_results.groupBy('client_name').count().withColumnRenamed('count', 'total_count')
results_with_zero2 = results_with_zero2.groupBy('client_name').count().withColumnRenamed('count', 'total_zero_count').withColumnRenamed('client_name', 'client_name_zero')
merged_test = merged_kpi_results2.join(results_with_zero2 , results_with_zero2.client_name_zero == merged_kpi_results2.client_name, how= 'left' )

print type (merged_test)
res= merged_test.select('client_name',( F.col('total_zero_count') / F.col('total_count'))* 100 ) .filter('total_zero_count is not null' )

res.show(100, False)
#todo write .all to one file 
# sults_with_zero.write.csv('/tmp/results_with_zero.csv')

<class 'pyspark.sql.dataframe.DataFrame'>
+-----------------------------------------------+----------------------------------------+
|client_name                                    |((total_zero_count / total_count) * 100)|
+-----------------------------------------------+----------------------------------------+
|promo                                          |6.60377358490566                        |
|secondary placement location visibility quality|90.85872576177285                       |
|PRODUCT_PRESENCE_IN_CATEGORY_CUSTOM            |17.38361508452536                       |
|store_assortment - SKU                         |17.366422142448364                      |
|eye_hand_level_sos                             |45.65476498937659                       |
|secondary placement location quality           |82.82548476454294                       |
|DST_BY_CATEGORY                                |2.9935275080906147                      |
|promo - SKU                                    

In [40]:
### test - uniqe resutls per kpi (there is only 2 result type for kpi)

test_results = merged_kpi_results.groupBy('client_name','result').count().withColumnRenamed('count', 'total_zero_count').withColumnRenamed('client_name', 'client_name_zero')
validation = merged_kpi_results2.join(test_results , test_results.client_name_zero == merged_kpi_results2.client_name, how= 'left' )

validation.select('client_name' ,'result' , (F.col('total_zero_count') / F.col('total_count'))).filter((F.col('total_zero_count') / F.col('total_count') ) > 0.5  ).show()


+--------------------+------+--------------------------------+
|         client_name|result|(total_zero_count / total_count)|
+--------------------+------+--------------------------------+
|secondary placeme...|   0.0|              0.9085872576177285|
|PRODUCT_PRESENCE_...| 100.0|              0.8261638491547464|
|store_assortment ...| 100.0|              0.8263357785755163|
|secondary placeme...|   0.0|              0.8282548476454293|
|         promo - SKU|   0.0|              0.6361589869855786|
+--------------------+------+--------------------------------+

